In [1]:
from choice_theory_figures import *

Note: figures labeled 'interactive' will not actually be interactive unless they are 
created in a live Jupyter notebook. If you're viewing this as a page on the course 
static website, you'll be able to see the default figures but not change the sliders.


# Discrete Choice Theory

Discrete choice modeling is at the heart of many transportion planning models. This
is because so many aspects of travel represent discrete choices: do I 
walk or drive?  Buy a car or not?  Take the freeway or side streets? Go to the store 
on the way to work?  Which store? 

All of these choices share some common features:  

- they are selections from a bunch of categorical options, 
- the categories often lack a natural ordering (walking is not clearly "more" or "less" than
  driving, just different), and
- choosing one option means not choosing the other options.

Beyond these factual aspects of the choices, we will also define some reasonable
assumptions about these choices, namely:

- the decision makers (the people traveling, generally) make **rational choices**
  from among the options, choosing whichever option they think is best, and
- those judgements about which option is best are based on the **attributes of 
  the alternatives**, and we as modelers can observe at least some of the 
  relevant attributes that decision makers are considering.

## Mathematical Derivation

We can convert these reasonable assumptions, plus a few more assumptions that
maybe are a little less reasonable, into a mathematical model to represent
the choice process.

Let's suppose that each of the alternatives in a choice problem has some value
associated with it, and that the decision maker makes their choice by selecting
the alternative with the highest value.  We'll call that value **"utility"**, which
then makes this decision making process **"utility maximization"**. 
Unfortunately, we can't actually observe utility, instead we can only observe
the actual or hypothetical choices that decision makers make.

Mathematically, decision maker $t$ chooses alternative $i$ from choices $C_t$ if and only if

$$
U_i \ge U_j \quad \forall j \in C_t.
$$

This is predicated on some formulation of utility

$$U_i = V_i + \varepsilon_i$$

where $V_i$ is a measured component, also called the *systematic utility*, and $\varepsilon_i$ is the the unobserved component, also called the *random utility*. Note that $\varepsilon_i$ isn't random from the perspective of the decision maker, it is only random from the perspective of the modeler. We can identify the probability of $t$ choosing alternative $i$ as

$$
\Pr_t(i | C_t) = \Pr(U_i \ge U_j, \forall j \in C_t) \\
= \Pr(V_i + \varepsilon_i \ge V_j + \varepsilon_j, \forall j \in C_t) \\
= \Pr(\varepsilon_j - \varepsilon_i \le V_i - V_j, \forall j \in C_t)
$$

In this last form we've grouped all the random utility together, which can be helpful.  If we can express these terms jointly, we have written this as a cumulative density function (CDF). Without making any particular assumption on the multivariate density function $f(\varepsilon)$ we can write the probability as an integral, like this:

$$
\Pr_t(i | C_t) = 
\int_{\varepsilon_i = -\infty}^{\infty} 
\int_{\varepsilon_j = -\infty}^{V_i - V_j + \varepsilon_i} 
f(\varepsilon) d\varepsilon_J d\varepsilon_{i-1} d\varepsilon_{i} 
$$

Particular assumptions about error distributions lead to particular model structures.

- If $\varepsilon$ has a multivariate normal distribution, we get a **Multinomial Probit** model. 
- If $\varepsilon$ has an IID Gumbel distribution, we get a **Multinomial Logit** model. **IID** means independent and identically distributed: each dimension has the same mean and variance, and there is no correlation across dimensions.

The Multinomial Probit is perfectly reasonable from a theoretical perspective, but it's inconvenient to work with in practice, because that integral in the equation above is messy, and the probit model doen't offer us a closed form analytic solution to get rid of it.  The Multinomial Logit, on the other hand, does have a bunch of really nice mathematical simplifications, which make it super easy to work with.  That's why you see logit models everywhere in transportation planning, and not probit models.

## The Gumbel Distribution

The normal distribution is (among other things) the 
distribution of the mean of a number of samples.  You've
likely heard of it before.  It's the default distribution
for a lot of things, and for good reasons.

The Gumbel distribution is much less well known.  It is
the (limit) distribution of the maximum (or the minimum) of 
a number of samples from a normal distribution.  We can show
it by taking a bunch of set of draws from a random normal distribution,
and taking the maximum in each set.

In [2]:
# 5 thousand repetitions of samples taken from a normal distribution
draws = [
    np.random.normal(size=[1,5000]),
    np.random.normal(size=[10,5000]),
    np.random.normal(size=[100,5000]),
    np.random.normal(size=[1000,5000]),
    np.random.normal(size=[10000,5000]),
    np.random.normal(size=[100000,5000]),
]  

In [3]:
gumbel_draws = [d.max(axis=0) for d in draws]

In [4]:
figure_gumbel_draws(draws)

<xmle.Elem 'div' with 1 children>

As the size of the pool from which the maximum is taken does up, so does the mean of the maximum.

In [5]:
np.mean(gumbel_draws, axis=1)

array([0.00841553, 1.53760621, 2.50080731, 3.23893381, 3.85007791,
       4.3742664 ])

If you want to work with gumbel distribution directly, you don't need to back
into it by actually taking the maximum of a bunch of things.
The gumbel distribution is available in the 
[`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/stats.html) package,
under the name [`gumbel_r`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gumbel_r.html) 
for the right-skewed version associated with the maximum, or the mirrored 
[`gumbel_l`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gumbel_l.html) for the left-skewed version associated
with the minimum.

In [6]:
from scipy.stats import gumbel_r, norm

In [7]:
figure_gumbel_draws(draws, gumbel_r)

<xmle.Elem 'div' with 1 children>

Similar to the normal distribution, the Gumbel distribution is defined by two parameters, one for position and one for width.  These are similar to the mean and standard deviation that define the normal distribution, although the defining parameters are not themselves the mean and standard deviation of the Gumbel.

The probability density function (pdf) for `gumbel_r` is:

$$f(x; \mu, \sigma) = \frac{1}{\sigma} \exp\left(-\frac{x+\mu}{\sigma}\right) \exp\left(-\exp\left(-\frac{x+\mu}{\sigma}\right)\right)$$

The cumulative density function (cdf) for `gumbel_r` is:

$$F(x; \mu, \sigma) = \exp\left(-\exp\left(-\frac{x-\mu}{\sigma}\right)\right)$$

The Gumbel distribution is sometimes referred to as a *type I Fisher-Tippett
distribution*. It is a bell-shaped distribution but obviously not "the" bell curve, which is generally the normal distribution. It is assymetric, but as we'll see below that is not important in the context of discrete choice analysis.

We can plot the PDF and CDF functions to get a better idea of the differences.

In [8]:
figure_distribution_vs_normal(gumbel_r)

<xmle.Elem 'div' with 1 children>

The mode (peak) of the standard Gumbel distribution is at zero,
but the mean is $0.577$, because the distribution is skewed.  The variance is $\frac{\pi^2}{6} = 1.645$.  

### Scale and Translation

Some properties of the Gumbel distribution include the ability to scale and translate (shift) it. If $\varepsilon \sim G(\mu,\sigma)$, then

- You can define a scaled version by multiplying by some constant $\alpha$ against both characteristic parameters:
  - $\alpha\varepsilon \sim Gumbel(\alpha\mu,\alpha\sigma)$
  - mode (peak) = $\alpha\mu$
  - mean = $\alpha\mu + \alpha\sigma 0.577$
  - variance = $\frac{\pi^2 \alpha^2 \sigma^2}{6}$

In [9]:
figure_gumbel_scale(
    gumbel_r(0,0.5),
    gumbel_r(0,1),
    gumbel_r(0,1.5),
    gumbel_r(0,2),
)

<xmle.Elem 'div' with 1 children>

- You can define a translated (shifted) version by adding only to the location parameter: 
  - ($\varepsilon + V) \sim Gumbel(\mu + V,\sigma)$ where $V$ is some non-random value
  - mode (peak) = $\mu + V$
  - mean = $\mu + V + \sigma 0.577$
  - variance = $\frac{\pi^2 \sigma^2}{6}$
  - outcome: the distribution can be shifted left or right with no change in shape or variance
  

In [10]:
figure_gumbel_translate(
    gumbel_r(0,1),
    gumbel_r(1,1),
    gumbel_r(2,1),
    gumbel_r(3,1),
)

<xmle.Elem 'div' with 1 children>

We can simplify the derivation of the Multinomial Logit model by
assuming $\mu = 0$ and $\sigma = 1$ (Standard Gumbel).
We will examine effect of different values later.

Up above, we created draws from a Gumbel distribution by generating
a lot of normally distributed random variables, and taking the maximum.
Scipy also allows directly generating random draws from the Gumbel
distribution (and 
[many other distributions](https://docs.scipy.org/doc/scipy/reference/stats.html)
also) using the `rvs` method.

In [11]:
gumbel_r(0,1).rvs()

1.9018508571928883

If we want more than one random draw from the same distribution, or
if we want to give a specific seed for reproducibility, we can give
`size` or `random_state` arguments, respectively.

In [12]:
gumbel_r(0,1).rvs(size=4, random_state=123)

array([ 1.01685242, -0.22416415, -0.39437711,  0.51843892])

## The Logistic Distribution

A handy property of the Gumbel distribution is that the difference between two independent 
Gumbel distributions (with same variance) is a **logistic distribution**.  

The probability density function is:

$$
\begin{align}
f(x; \mu, \sigma)  & = \frac{e^{-(x-\mu)/\sigma}} {s\left(1+e^{-(x-\mu)/\sigma}\right)^2} \\[4pt]
\end{align}
$$

The cumulative density function is:

$$
F(x; \mu, \sigma) = \frac{1}{1+\exp({-(x-\mu)/\sigma})}
$$

It looks like this:

In [13]:
from scipy.stats import logistic
figure_distribution_vs_normal(logistic)

<xmle.Elem 'div' with 1 children>

Unlike the Gumbel, the Logistic distribution is symmetric.  Since it is also
unimodal (one hump) the mean and mode (peak) are at the same value.  When it
is created by taking the difference between two IID Gumbels, this value is 
equal to the difference between the location parameters (peaks) of those
two Gumbels.

$\varepsilon_1 \sim Gumbel(V_1, 1)$

$\varepsilon_2 \sim Gumbel(V_2, 1)$

$\varepsilon_1 - \varepsilon_2 = \varepsilon_3 \sim Logistic(V_1-V_2, 1)$


## Maximium over Multiple Gumbels

Suppose we have two random variables, both with a Gumbel distribution, both with the same variance.

$$\varepsilon_1 \sim G(V_1, 1)$$

$$\varepsilon_2 \sim G(V_2, 1)$$

Then, the distribution of the maximum, i.e. $\max(\varepsilon_1, \varepsilon_2)$ is *also* a Gumbel distribution, and *also* with the same variance.  

$$\max(\varepsilon_1, \varepsilon_2) \sim G(V_\star, 1)$$

with $V_\star = \log\left(\exp(V_1)+\exp(V_2)\right)$.

By extension, the same holds for the maximum of any set of $N$ Gumbel distributions:

$$V_\star = \log\left(\sum_{j=1}^{N}\exp(V_j)\right)$$.

For obvious reasons, this term is called a "logsum". You will find that logsums are used in many places when working with discrete choice models.

## Deriving the Multinomial Logit Choice Model

Recall our utility formulation.  The probability of a decision maker choosing a particular alternative $i$ is equal to the probability that the utility $U_i$ is greater than or equal to the utility $U_j$ for all other possible choices $j$.  This is equivalent to writing:

$$
\Pr\left(U_i \ge \max(U_j, \forall j \ne i)\right)
$$

or

$$
\Pr\left(V_i + \varepsilon_i \ge \max(V_j + \varepsilon_j, \forall j \ne i)\right)
$$

or

$$
\Pr\left(V_i \ge \max(V_j + \varepsilon_j, \forall j \ne i) - \varepsilon_i\right)
$$


Some important features of this way of writing the probabilities:

- On the left side of the inequality we have just $V_i$ which is not a random variable.
- On the right side we have $\max(V_j + \varepsilon_j, \forall j \ne i)$ which is 
  a maximum of (shifted) Gumbel distributions, which then is itself a Gumbel distribution,
  let's call it $\varepsilon_\star$ which has a (shifted) location of $V_\star$.
- Then we take the difference of that Gumbel distribution and another (the one 
  for $\varepsilon_i$), which makes one logistic distribution.
- That one logistic distribution is the only term on the right side of the inequality.
- We therefore reduced our multi-dimensional integral from before to a one-dimensional
  integral on a single logistic distribution ... which conveniently has a closed form solution:
  the CDF:
  
$$
\Pr\left(V_i \ge \max(V_j + \varepsilon_j, \forall j \ne i) - \varepsilon_i\right) = 
\frac{1}{1+\exp(V_\star - V_i)}
$$

We can reformat this a bit to make it more obviously symmetric and generalizable: 

$$
\frac{1}{1+\exp(V_k - V_i)}
= \frac{1}{1+\exp(V_k)/\exp(V_i)} = \frac{\exp(V_i)}{\exp(V_i)+\exp(V_k)}
$$

And recall that $V^\star = \log\left(\sum_{j=1}^{N}\exp(V_j)\right)$, which leads to

$$
\Pr(i) = \frac{\exp(V_i)}{\sum_{j=1}^{N}\exp(V_j)}
$$

## Arbitrary Scale 

Consider what happens if set the scale of the Gumbel distributions in our utility
functions to some value other than 1:

The utility function for alternative $j$ is now

$$
U_j = V_j + \varepsilon_j,\quad \varepsilon_j \sim Gumbel(0,\mu)
$$



Let's take this but define a different scaled measure of utility $U^\prime$.

$$
\begin{align}
U^\prime_j &= \mu U_j \\
&= \mu (V_j + \varepsilon_j),\quad \varepsilon_j \sim Gumbel(0,1) \\
&=  \mu V_j + \mu\varepsilon_j,\quad \mu\varepsilon_j \sim Gumbel(0,\mu)
\end{align}
$$

You can see that the two different models are practically the same, except for the scale of the variance.

However, because we will be choosing the functional form of $V$ to best fit the data, we can scale the
values we get for $V$ and end up with an identical model with respect to the output probabilities.

If we were able to observe the *utility* values directly, we could identify a unique scale of the 
$\varepsilon$ terms that would fit best.  But because we can only observe the choices, we cannot
actually identify which scale is better, and we are free to choose any scale factor that is mathematically
convenient (i.e., 1).

## Arbitrary Location 


Consider what happens if set the location (mode) of the Gumbel distributions in our utility
functions to some value other than 0:

The utility function for alternative $j$ is now

$$
U_j = V_j + \varepsilon_j,\quad \varepsilon_j \sim Gumbel(\delta, 1)
$$

Let's take this but define a different shifted measure of utility $U^\prime$.

$$
\begin{align}
U^\prime_j &= V_j + \varepsilon_j,\quad \varepsilon_j \sim Gumbel(\delta, 1) \\
&= V_j + \delta + \varepsilon_j,\quad \varepsilon_j \sim Gumbel(0,1)
\end{align}
$$

As for the scale, we can push an arbitrary location adjustment from $\varepsilon$ into $V$, and end up with the same model with respect to probabilities.

## Too Much Arbitrary-ness in Location

Consider what happens if we introduce an arbitrary constant shift in utility for *every* alterantive.

What happens to the differences in utilities between alternatives?  

$$
V_i^\prime - V_j^\prime = (V_i + \delta) - (V_j + \delta) = V_i  - V_j 
$$

What happens to the probabilities of alternatives?  


$$
\Pr(i) = \frac{\exp(V_i + \delta)}{\sum_{j=1}^{N}\exp(V_j + \delta)}
= \frac{\exp(V_i)\exp(\delta)}{\sum_{j=1}^{N}\exp(V_j)\exp(\delta)}
= \frac{\exp(\delta)}{\exp(\delta)}\frac{\exp(V_i)}{\sum_{j=1}^{N}\exp(V_j)}
= \frac{\exp(V_i)}{\sum_{j=1}^{N}\exp(V_j)}
$$

- We can shift constants for all alternatives the same and get the same probability model.
- We pick any alternative arbitrarily and set the constant to zero and get the same probability model.
- **Only the differences in utility matter, not the absolute values.**

## Typical Form of Systematic Utility

In most transportation planning applications, the systematic utility is given as a **linear in parameters** function.

This means that the functional form of $V_{ti}$ for decision maker $t$ and alternative $i$ looks like

$$
V_{ti} = X_{ti} \beta_{i} = \sum_{k=1}^{K} X_{tik} \beta_{ik}
$$

You may recall from the `statsmodel` package that in order to include a y-intercept or constant term in the model, it was necessary to explicitly add a column of all 1's to the dataframe.  The same is generally true when expressing the utility function for a discrete choice model like this.

One important feature to keep track of here that is different from an ordinary least squares linear regression model: **only the differences in utility matter**.

This means that *something* needs to induce some differences in the systematic utility between alternatives.  That *something* can be:

- differences in the observed $X$ data values (e.g. the travel time is different for different modes), or
- differences in the $\beta$ parameter values across modes, or
- both.

It is important to pay attention to instances where observed data values do *not* vary across modes, and ensure
that the parameters *do* vary in those instances.  These are sometimes called alternative-specific variables.

(Yes, this is ironic, because the variables themselves are in fact not specific to the alternative.)

## Independence of Irrelevant Alternatives (IIA)

One important property of the MNL model is "Independence of Irrelevant Alternatives".

This refers to the fact that the ratio of choice probabilities between pairs of alternatives is independent of the availability or attributes of other alternatives.

$$
\frac{\Pr(i)}{\Pr(k)} = \frac{\exp(V_i)}{\sum_j \exp(V_j)}\frac{\sum_j \exp(V_j)}{\exp(V_k)}
=\frac{\exp(V_i)}{\exp(V_k)} = \exp(V_i - V_k)
$$


How reasonable is this outcome?  In a lot of cases, not very reasonable at all.  In a later section, we'll look at the "nested multinomial logit" or "nested logit" model, which solves some of these issues.